# Time Series Analysis: "The Final Project"

`End? No, the journey doesn't end here. Death is just another path. One that we all must take.
-J.R.R. Tolkien, The Return of the King`

---

## Libraries

In [1]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
# from tsa_functions import *

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)


In [2]:
from tsa_tools import *

---

## M5 Forecasting

For this "final project", we will be forecasting the <b><u>level 9</b></u> series (unit sales of all products, aggregated for each store and department).

Load `sales_train_evaluation.csv` and use observations from `d_1 to d_1913` for training and `d_1914 to d_1941` for testing.

In [3]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_sales = pd.read_csv('../data/m5/sales_train_evaluation.csv')
display(df_calendar, df_sales)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [4]:
design_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[:-28])
test_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[-28:])
display(design_df, test_df)

state_id                              CA                                \
store_id                            CA_1                                 
cat_id                           HOBBIES                                 
dept_id                        HOBBIES_1                                 
item_id                    HOBBIES_1_001                 HOBBIES_1_002   
id         HOBBIES_1_001_CA_1_evaluation HOBBIES_1_002_CA_1_evaluation   
date                                                                     
2011-01-29                             0                             0   
2011-01-30                             0                             0   
2011-01-31                             0                             0   
2011-02-01                             0                             0   
2011-02-02                             0                             0   
..                                   ...                           ...   
2016-04-20                             1                             1   
2016-04-21                             3                             0   
2016-04-22                             0                             0   
2016-04-23                             1                             0   
2016-04-24                             1                             0   

state_id                                                                \
store_id                                                                 
cat_id                                                                   
dept_id                                                                  
item_id                    HOBBIES_1_003                 HOBBIES_1_004   
id         HOBBIES_1_003_CA_1_evaluation HOBBIES_1_004_CA_1_evaluation   
date                                                                     
2011-01-29                             0                             0   
2011-01-30                             0                             0   
2011-01-31                             0                             0   
2011-02-01                             0                             0   
2011-02-02                             0                             0   
..                                   ...                           ...   
2016-04-20                             1                             0   
2016-04-21                             0                             1   
2016-04-22                             1                             3   
2016-04-23                             1                             7   
2016-04-24                             1                             2   

state_id                                                                \
store_id                                                                 
cat_id                                                                   
dept_id                                                                  
item_id                    HOBBIES_1_005                 HOBBIES_1_006   
id         HOBBIES_1_005_CA_1_evaluation HOBBIES_1_006_CA_1_evaluation   
date                                                                     
2011-01-29                             0                             0   
2011-01-30                             0                             0   
2011-01-31                             0                             0   
2011-02-01                             0                             0   
2011-02-02                             0                             0   
..                                   ...                           ...   
2016-04-20                             1                             0   
2016-04-21                             2                             0   
2016-04-22                             2                             2   
2016-04-23                             2                             0   
2016-04-24                             4                             0   

s

state_id                              CA                                \
store_id                            CA_1                                 
cat_id                           HOBBIES                                 
dept_id                        HOBBIES_1                                 
item_id                    HOBBIES_1_001                 HOBBIES_1_002   
id         HOBBIES_1_001_CA_1_evaluation HOBBIES_1_002_CA_1_evaluation   
date                                                                     
2016-04-25                             0                             0   
2016-04-26                             0                             1   
2016-04-27                             0                             0   
2016-04-28                             2                             0   
2016-04-29                             0                             0   
2016-04-30                             3                             0   
2016-05-01                             5                             0   
2016-05-02                             0                             0   
2016-05-03                             0                             0   
2016-05-04                             1                             1   
2016-05-05                             1                             0   
2016-05-06                             0                             0   
2016-05-07                             2                             0   
2016-05-08                             1                             0   
2016-05-09                             2                             0   
2016-05-10                             2                             0   
2016-05-11                             1                             0   
2016-05-12                             0                             0   
2016-05-13                             2                             0   
2016-05-14                             4                             1   
2016-05-15                             0                             2   
2016-05-16                             0                             1   
2016-05-17                             0                             1   
2016-05-18                             0                             0   
2016-05-19                             3                             0   
2016-05-20                             3                             0   
2016-05-21                             0                             0   
2016-05-22                             1                             0   

state_id                                                                \
store_id                                                                 
cat_id                                                                   
dept_id                                                                  
item_id                    HOBBIES_1_003                 HOBBIES_1_004   
id         HOBBIES_1_003_CA_1_evaluation HOBBIES_1_004_CA_1_evaluation   
date                                                                     
2016-04-25                             0                             0   
2016-04-26                             0                             0   
2016-04-27                             1                             1   
2016-04-28                             1                             2   
2016-04-29                             0                             4   
2016-04-30                             2                             1   
2016-05-01                             1                             6   
2016-05-02                             0                             4   
2016-05-03                             0                             0   
2016-05-04                             0                             0   
2016-05-05                             0                             0   
2016-05-06                             2                             2   
201

In [6]:
design_df.sum(axis=1, level=["state_id", "item_id"])

state_id              CA                                            \
item_id    HOBBIES_1_001 HOBBIES_1_002 HOBBIES_1_003 HOBBIES_1_004   
date                                                                 
2011-01-29             0             0             0             7   
2011-01-30             0             0             0             4   
2011-01-31             0             0             0             4   
2011-02-01             0             0             0            10   
2011-02-02             0             0             0             4   
...                  ...           ...           ...           ...   
2016-04-20             7             1             1             3   
2016-04-21             4             0             0             5   
2016-04-22             1             2             5            11   
2016-04-23             8             3             3            23   
2016-04-24             7             1             2             6   

state_id                                                            \
item_id    HOBBIES_1_005 HOBBIES_1_006 HOBBIES_1_007 HOBBIES_1_008   
date                                                                 
2011-01-29             0             0             0            23   
2011-01-30             0             0             0            27   
2011-01-31             0             0             0            33   
2011-02-01             0             0             0            13   
2011-02-02             0             0             0            53   
...                  ...           ...           ...           ...   
2016-04-20             1             2             3            26   
2016-04-21             4             1             2            33   
2016-04-22             5             3             0            26   
2016-04-23             9             2             3            26   
2016-04-24             9             8             1            20   

state_id                                ...          WI              \
item_id    HOBBIES_1_009 HOBBIES_1_010  ... FOODS_3_818 FOODS_3_819   
date                                    ...                           
2011-01-29             7             3  ...           0          23   
2011-01-30             2             0  ...           0          14   
2011-01-31            13             5  ...           0           8   
2011-02-01             6             1  ...           0           7   
2011-02-02             5             2  ...           0           5   
...                  ...           ...  ...         ...         ...   
2016-04-20             2             0  ...          13           8   
2016-04-21             0             1  ...           7           5   
2016-04-22             0             5  ...           7           6   
2016-04-23             0             6  ...           9           5   
2016-04-24             6             5  ...           5          15   

state_id                                                                \
item_id    FOODS_3_820 FOODS_3_821 FOODS_3_822 FOODS_3_823 FOODS_3_824   
date                                                                     
2011-01-29           6           0           9           0           2   
2011-01-30           3           0          10           0           0   
2011-01-31           6           0           6           2           0   
2011-02-01           2           0          10           2           0   
2011-02-02           1           0           5           0           1   
...                ...         ...         ...         ...         ...   
2016-04-20           8           0           3           1           0   
2016-04-21          10           0          12           1           3   
2016-04-22          10           0           9           2           0   
2016-04-23           5           6           5           0           2   
2016-04-24           7           2          10           1           1   

state_

---

## Part 1. Baseline Methods (10 pts.)

### Q1. (10 pts.)

Extract all level 9 series from the dataset.

For each series, generate a 28-step forecast using the methods enumerated below and calculate the `RMSSE` against the test set:

1. `Naive`


2. `Seasonal Naive`


3. `SES`


4. `Holt's Linear`


5. `Additive Holt-Winters`

Summarize the metrics in a dataframe and print it.

---

## Part 2. LightGBM (30 pts.)

### Q2. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a one-step recursive forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [ ]:
# Your code here

### Q3. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a direct forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [ ]:
# Your code here

### Q4. (10 pts.)

For all series, generate a 28-step forecast by combining the forecasts generated by the models in Q2 and Q3 (i.e. simple averaging).

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [ ]:
# Your code here

---

## Part 3. WRMSSE (10 pts.)

### Q5.  (10 pts.)

Calculate the `WRMSSE` for the all the methods described above. The weights can be found in `weights_validation.csv`.

For reference, the M5 benchmarks have the following `WRMSSE` scores at level 9:

- `Naive` = <b>1.764</b>


- `S.Naive` = <b>0.888</b>


- `ES_bu` = <b>0.728</b>

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
# Your code here

---

## Part 4. Middle-Out Method (30 pts.)

### Q6. Bottom-Up (15 pts.)

Using your forecasts from the best performing method in Q5, use the bottom-up method described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 1 to 8.

Calculate the `WRMSSE` for levels 1 to 8 against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
# Your code here

### Q7. Top-Down  (15 pts.)

Using your forecasts from the best performing method in Q5, use the top-down method with `average historical proportions` described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 10 to 12.

Calculate the `WRMSSE` for levels 10 to 12  against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
# Your code here